In [1]:
import tkinter as tk
import random

class TicTacToe:
    def __init__(self, master):
        self.master = master
        self.master.title("Tic Tac Toe")
        self.current_player = "X"
        self.buttons = [[None for _ in range(3)] for _ in range(3)]
        self.board = [["" for _ in range(3)] for _ in range(3)]
        self.stats = {"X": 0, "O": 0, "Draws": 0}
        self.create_widgets()

    def create_widgets(self):
        top = tk.Frame(self.master)
        top.pack()

        self.mode = tk.StringVar(value="2P")
        tk.Radiobutton(top, text="2 Player", variable=self.mode, value="2P", command=self.reset_board).pack(side=tk.LEFT)
        tk.Radiobutton(top, text="vs AI", variable=self.mode, value="AI", command=self.reset_board).pack(side=tk.LEFT)
        tk.Radiobutton(top, text="Computer vs Computer", variable=self.mode, value="AIvsAI", command=self.reset_board).pack(side=tk.LEFT)

        self.ai_enabled = tk.BooleanVar(value=True)
        tk.Checkbutton(top, text="Enable AI", variable=self.ai_enabled, command=self.reset_board).pack(side=tk.LEFT)

        tk.Button(top, text="Play Again", command=self.reset_board).pack(side=tk.LEFT)

        self.stats_label = tk.Label(self.master, text="", font=("Arial", 14))
        self.stats_label.pack()

        self.frame = tk.Frame(self.master)
        self.frame.pack()

        for i in range(3):
            for j in range(3):
                button = tk.Button(self.frame, text="", font=("Arial", 24), width=5, height=2,
                                   command=lambda row=i, col=j: self.on_click(row, col))
                button.grid(row=i, column=j)
                self.buttons[i][j] = button

        self.update_stats()
        if self.mode.get() == "AIvsAI":
            self.master.after(500, self.auto_ai_loop)

    def on_click(self, row, col):
        if self.board[row][col] == "" and not self.check_winner(self.board):
            self.board[row][col] = self.current_player
            self.buttons[row][col].config(text=self.current_player)

            winner = self.check_winner(self.board)
            if winner:
                self.end_game(winner)
            else:
                self.switch_player()
                if self.mode.get() == "AI" and self.current_player == "O" and self.ai_enabled.get():
                    self.master.after(500, self.ai_move)

    def ai_move(self):
        if self.check_winner(self.board):
            return
        empty_cells = [(i, j) for i in range(3) for j in range(3) if self.board[i][j] == ""]
        if empty_cells:
            row, col = random.choice(empty_cells)
            self.on_click(row, col)

    def auto_ai_loop(self):
        if not self.check_winner(self.board) and any("" in row for row in self.board):
            self.ai_move()
            self.master.after(500, self.auto_ai_loop)

    def switch_player(self):
        self.current_player = "O" if self.current_player == "X" else "X"

    def check_winner(self, board):
        for i in range(3):
            if board[i][0] == board[i][1] == board[i][2] != "":
                return board[i][0]
            if board[0][i] == board[1][i] == board[2][i] != "":
                return board[0][i]
        if board[0][0] == board[1][1] == board[2][2] != "":
            return board[0][0]
        if board[0][2] == board[1][1] == board[2][0] != "":
            return board[0][2]
        if all(cell != "" for row in board for cell in row):
            return "Draw"
        return None

    def end_game(self, winner):
        if winner == "Draw":
            self.stats["Draws"] += 1
        else:
            self.stats[winner] += 1
        self.update_stats()
        for i in range(3):
            for j in range(3):
                self.buttons[i][j].config(state=tk.DISABLED)

    def update_stats(self):
        stats_text = f"X Wins: {self.stats['X']} | O Wins: {self.stats['O']} | Draws: {self.stats['Draws']}"
        self.stats_label.config(text=stats_text)

    def reset_board(self):
        self.board = [["" for _ in range(3)] for _ in range(3)]
        self.current_player = "X"
        for i in range(3):
            for j in range(3):
                self.buttons[i][j].config(text="", state=tk.NORMAL)
        if self.mode.get() == "AIvsAI":
            self.master.after(500, self.auto_ai_loop)

# Run the game
if __name__ == "__main__":
    root = tk.Tk()
    game = TicTacToe(root)
    root.mainloop()
